In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
! wget -w -O - ipv4.icanhazip.com

wget: --wait: Invalid time period ‘-O’


In [ ]:
!pip install pdf2image
!pip install pytesseract

In [ ]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,703 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (260 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121965 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 49.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
import fitz
import pdf2image
import pytesseract
from PIL import Image


st.set_page_config(page_title='PDF2Texto', page_icon=":tada:", layout="wide")
# HEADER
with st.container():
    st.title(":violet[_PDF2TEXTO_] :sparkles:")
    st.subheader(""":grey[Welcome to _your PDF to text converter_ for FREE]""", divider="orange")

def read_pdf(pdf_file):
    # Open the PDF file
    try:
        doc = fitz.open(stream= pdf_file.read(), filetype = "pdf")
        text = ""
    except Exception as e:
        print(f"ERROR while reading opening pdf with fitz: {e}")
        return None

    # Iterate through each page of the PDF
    for page in doc:
        # Extract text from the current page
        text += page.get_text()

    # Close the document
    doc.close()
    return text


def ocr_for_pdf(file):
  st.info("OCR loading...")
  file.seek(0) # reset file pointer
  images = pdf2image.convert_from_bytes(file.read())

  complete_text = []
  counter_pages=0
  for img, page in enumerate(images):
    counter_pages = counter_pages +1
    st.write("Page number:", counter_pages)
    #st.image(page, use_column_width = True) # display image
    text= pytesseract.image_to_string(page) # read text using pytesseract
    complete_text.append(text)
    num_pages = len(images)
  return complete_text, num_pages



# Upload a file
def load_file():
    file = st.file_uploader(":grey[Select your pdf file and... let's go!]", type= ["pdf"], accept_multiple_files=False)
    if file is not None:
        try:
            # Read and print the text from the PDF
            pdf_text = read_pdf(file)
            if pdf_text:
                #st.write(pdf_text)
                download_txt_converted_file(pdf_text)

            else:
                st.write("OCR is required. The process may take longer")
                final_text, number_pages = ocr_for_pdf(file)
                final_text_str = "\n".join(final_text) # output of ocr function is list, transform to string
                #st.write(final_text)
                download_txt_converted_file(final_text_str)

        except Exception as e:
            st.error(f"An error occurred while reading the PDF: {e}")
            st.warning("OOOooops! Your file seems to be empty\n Try with another pdf")

def download_txt_converted_file(text_content):
    if text_content:
        st.download_button("Download your text content!", text_content, file_name="pdf2texto_converted.txt")
        st.write("Do you want to translate this text to another? See Language2Language website here")
        st.write("\n\n:violet[Thank you! :confetti_ball:]")


load_file()

Writing app.py


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501